# **Computational Theory Assessment (2024/2025)**
Ronan Francis (G00403092)
Computational Theory

## **Table of Contents**
1. [Task 1: Binary Representations](#task-1-binary-representations)
2. [Task 2: Hash Functions](#task-2-hash-functions)
3. [Task 3: SHA256 Padding](#task-3-sha256-padding)
4. [Task 4: Prime Numbers](#task-4-prime-numbers)
5. [Task 5: Roots](#task-5-roots)
6. [Task 6: Proof of Work](#task-6-proof-of-work)
7. [Task 7: Turing Machines](#task-7-turing-machines)
8. [Task 8: Computational Complexity](#task-8-computational-complexity)

## **Task 1: Binary Representations**

[Circular Shift](https://en.wikipedia.org/wiki/Circular_shift)

[Bitwise Operators](https://www.geeksforgeeks.org/python-bitwise-operators/)

### 1. [rotl(x, n=1)](https://en.cppreference.com/w/cpp/numeric/rotl)
The function `rotl(x, n=1)` that rotates the bits in a 32-bit unsigned integer to the left `n` places.

In [58]:
def rotl(x, n=1):
    # How many bits we need to represent x (at least 1)
    width = x.bit_length() or 1
    mask = (1 << width) - 1 # For example if width is 7, mask is 0b1111111

    # Make sure n is in the range [0, width]
    n %= width

    # Do the rotation:
    # - shift x to the left by n bits
    # - bitwise AND with the mask to keep only the width least significant bits
    # - bitwise OR with the bits that were "shifted out" to the left
    return ((x << n) & mask) | (x >> (width - n))

# Example
x = 0b1010101 # 85
rotated = rotl(x) # rotate left by 1

# Print the binary representation of x and the rotated value, with leading zeros
width = x.bit_length() or 1
print("Original:", format(x, '0{}b'.format(width)))
print("Rotated: ", format(rotated, '0{}b'.format(width)))


Original: 1010101
Rotated:  0101011


### 2. [rotr(x, n=1)](https://en.cppreference.com/w/cpp/numeric/rotr)
The function `rotr(x, n=1)` that rotates the bits in a 32-bit unsigned integer to the right `n` places.

In [59]:
def rotr(x, n=1):
    width = x.bit_length() or 1 # How many bits we need to represent x (at least 1)
    mask = (1 << width) - 1  # For example if width is 7, mask is 0b1111111
    x &= mask # Make sure x is in the range [0, 2**width)
    n %= width # Make sure n is in the range [0, width)
    return (x >> n) | ((x << (width - n)) & mask) # Do the rotation

# Example
x = 0b0101010 # 42
rotated = rotl(x) # rotate left by 1

# Print the binary representation of x and the rotated value, with leading zeros
width = x.bit_length() or 1
print("Original:", format(x, '0{}b'.format(width)))
print("Rotated: ", format(rotated, '0{}b'.format(width)))

Original: 101010
Rotated:  010101


### 3. [ch(x, y, z)](https://crypto.stackexchange.com/questions/5358/what-does-maj-and-ch-mean-in-sha-256-algorithm)
The function `ch(x, y, z)` that chooses the bits from `y` where `x` has bits set to `1` and bits in `z` where `x` has bits set to `0`.

In [60]:
def ch(x, y, z):
    # (x AND y) XOR (NOT x AND z)
    return (x & y) ^ (~x & z)

# Example
x = 0b1010 # 10
y = 0b0000 # 0  
z = 0b1111 # 15

result = ch(x, y, z)
print("Expeted:", bin(y))
print("Result:", bin(result))

Expeted: 0b0
Result: 0b101


### 4. [maj(x, y, z)](https://crypto.stackexchange.com/questions/5358/what-does-maj-and-ch-mean-in-sha-256-algorithm)

The function `maj(x, y, z)` which takes a majority vote of the bits in `x`, `y`, and `z`.  
The output should have a `1` in bit position `i` where at least two of `x`, `y`, and `z` have `1`'s in position `i`.  
All other output bit positions should be `0`.

In [61]:
def maj(x, y, z):
    # (x AND y) XOR (x AND z) XOR (y AND z)
    return (x & y) ^ (x & z) ^ (y & z)

# Example
x = 0b1010 # 10
y = 0b0000 # 0
z = 0b1111 # 15

print("Expected:", bin(x))
print("Result:  ", bin(maj(x, y, z)))

Expected: 0b1010
Result:   0b1010


[Back to Top](#table-of-contents)

## **Task 2: Hash Functions**

[`Ord()` Function](https://www.w3schools.com/python/ref_func_ord.asp)

[Hash Size](https://www.cimat.mx/ciencia_para_jovenes/bachillerato/libros/%5BKernighan-Ritchie%5DThe_C_Programming_Language.pdf)

In [62]:
def hash(s, base=31, modulus=101):
    hashValue = 0
    for c in s:
        hashValue = ord(c) + base * hashValue # ord(c) returns the ASCII value of c
    return hashValue % modulus 

In [63]:
# Some test strings
test_strings = [
    "hello", "world", "foo", "bar", "baz", "hash", "collision", "test",
    "abcd", "abce", "abcf", "java", "python", "rust", "c++", "javascript"
]

for string in test_strings:
    print(f"{string!r} -> {hash(string):>08b}, {hash(string)}")

'hello' -> 00010001, 17
'world' -> 00100010, 34
'foo' -> 01000101, 69
'bar' -> 00100100, 36
'baz' -> 00101100, 44
'hash' -> 00001111, 15
'collision' -> 00001000, 8
'test' -> 01010110, 86
'abcd' -> 01100100, 100
'abce' -> 00000000, 0
'abcf' -> 00000001, 1
'java' -> 01011101, 93
'python' -> 01011011, 91
'rust' -> 00010001, 17
'c++' -> 00111100, 60
'javascript' -> 00101011, 43


*Collison in `rust -> 17` and `hello -> 17`*

In [64]:
import string
import random

def generate_unique_strings(count, length=8):
    """Generate a set of unique random alphanumeric strings of given length."""
    unique_strings = set()
    
    while len(unique_strings) < count:
        new_string = ''.join(random.choices(string.ascii_letters + string.digits, k=length))
        unique_strings.add(new_string)
    
    return list(unique_strings)

def test_collisions(strings, bases, moduli):
    """
    strings:  list of test strings to hash
    bases:    list of possible 'base' values
    moduli:   list of possible 'modulus' values
    
    Returns a dictionary with collision statistics for each (base, modulus) pair.
    """
    
    results = {}
    
    for base in bases:
        for mod in moduli:
            # Dictionary to keep track of hash -> list of strings
            hash_map = {}
            
            for s in strings:
                h = hash(s, base, mod)
                if h not in hash_map:
                    hash_map[h] = [s]
                else:
                    hash_map[h].append(s)
            
            # Count collisions by counting the number of strings in each bucket
            collision_count = sum(len(lst) - 1 for lst in hash_map.values() if len(lst) > 1)
            total_strings = len(strings) 
            
            results[(base, mod)] = {
                "collision_count": collision_count,
                "collision_rate": collision_count / total_strings if total_strings else 0,
                "hash_map": hash_map
            }
    
    return results

In [65]:
# Example test strings
test_strings = generate_unique_strings(1000, 8)
    
# Possible bases and moduli to test
candidate_bases = [31, 144, 257]
candidate_moduli = [101, 5_054, 10_007 ] 
    
# Run tests
collision_results = test_collisions(test_strings, candidate_bases, candidate_moduli)
    
# Print summary
for (base, mod), stats in collision_results.items():
    if (base, mod) == (31, 101):
        print("*", end =" ")
    print(f"hash(s,{base},{mod}) -> Collisions: {stats['collision_count']}, "
        f"Collision Rate: {stats['collision_rate']:.2f}")

* hash(s,31,101) -> Collisions: 899, Collision Rate: 0.90
hash(s,31,5054) -> Collisions: 82, Collision Rate: 0.08
hash(s,31,10007) -> Collisions: 48, Collision Rate: 0.05
hash(s,144,101) -> Collisions: 899, Collision Rate: 0.90
hash(s,144,5054) -> Collisions: 91, Collision Rate: 0.09
hash(s,144,10007) -> Collisions: 46, Collision Rate: 0.05
hash(s,257,101) -> Collisions: 899, Collision Rate: 0.90
hash(s,257,5054) -> Collisions: 99, Collision Rate: 0.10
hash(s,257,10007) -> Collisions: 46, Collision Rate: 0.05


### Why `31` and `101`
After testing a range of different numbers from `[31, 37, 257]` for the bases and `[101, 127, 10_007]` 

from Joshua Bloch, [Effective Java](https://ia800308.us.archive.org/16/items/java_20230528/Joshua%20Bloch%20-%20Effective%20Java%20%283rd%29%20-%202018.pdf),
> The value 31 was chosen because it is an odd prime. If it were even and the multiplication overflowed, information would be lost, because multiplication by 2 is equivalent to shifting. The advantage of using a prime is less clear, but it is traditional. 
> A nice property of 31 is that the multiplication can be replaced by a shift and a subtraction for better performance on some architectures: 31 * i == (i << 5) - i. Modern VMs do this sort of optimization automatically

The same can be said about 101 as well

[Back to Top](#table-of-contents)

## **Task 3: SHA256**
...

[Back to Top](#table-of-contents)

## **Task 4: Prime Numbers**
...

[Back to Top](#table-of-contents)

## **Task 5: Roots**
...

[Back to Top](#table-of-contents)

## **Task 6: Proof of Work**
...

[Back to Top](#table-of-contents)

## **Task 7: Turing Machines**
...

[Back to Top](#table-of-contents)

## **Task 8: Computational Complexity**
...

[Back to Top](#table-of-contents)

## **Conclusion**
...

[Back to Top](#table-of-contents)